<a href="https://colab.research.google.com/github/lunecarvalho/similarity-sentences-nlp/blob/main/similarity_sentences_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://raw.githubusercontent.com/alura-cursos/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [ ]:
import pandas as pd

dados = pd.read_csv(url)
dados

,nota_review,review
0,5,Recebi bem antes do prazo estipulado.
1,5,Parabéns lojas Zoop adorei comprar pela Intern...
2,4,aparelho eficiente. no site a marca do aparelh...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...
39458,3,"O produto não foi enviado com NF, não existe v..."
39459,5,"Excelente mochila, entrega super rápida. Super..."
39460,1,Solicitei a compra de uma capa de retrovisor c...


In [ ]:
dados['review'][1]

'Parabéns lojas Zoop adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [ ]:
dados['review'][8]

'A compra foi realizada facilmente.\r\nA entrega foi efetuada muito antes do prazo dado.\r\nO produto já começou a ser usado e até o presente,\r\nsem problemas.'

In [ ]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [ ]:
dados['review'][18]

'otimo vendedor chegou ate antes do prazo , adorei o produto'

In [ ]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language='portuguese')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 58.1 MB/s eta 0:00:00


In [ ]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences = dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [ ]:
modelo.wv.most_similar('entrega')

[('Entrega', 0.7579216361045837),
 ('entregue', 0.7184001207351685),
 ('compra', 0.7105124592781067),
 ('chegou', 0.6817407608032227),
 ('entregou', 0.6612473130226135),
 ('preparar', 0.6325778365135193),
 ('envio', 0.6313653588294983),
 ('sorriso', 0.6210829615592957),
 ('Chegou', 0.6198225021362305),
 ('mercadoria', 0.6146730780601501)]

In [ ]:
modelo.wv.most_similar('vendedor')

[('atendimento', 0.8572612404823303),
 ('serviço', 0.8074656128883362),
 ('fornecedor', 0.7500720024108887),
 ('veiculo', 0.7239760756492615),
 ('produto', 0.7152891755104065),
 ('processo', 0.7085978388786316),
 ('parceiro', 0.7057496905326843),
 ('cumpriu', 0.6909526586532593),
 ('trabalho', 0.6848700642585754),
 ('cumpriram', 0.6713030934333801)]

In [ ]:
modelo.wv.most_similar('produto')

[('pedido', 0.749972939491272),
 ('relógio', 0.7460570335388184),
 ('vendedor', 0.7152891755104065),
 ('endereço', 0.7050330638885498),
 ('Produto', 0.6978251934051514),
 ('mesmo', 0.6896668076515198),
 ('tempo', 0.6797294616699219),
 ('Mas', 0.6640905737876892),
 ('mercadoria', 0.6585670113563538),
 ('condizia', 0.6553379893302917)]

In [ ]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [ ]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in dados.iterrows()]

In [ ]:
dados_tag

[TaggedDocument(words=['Recebi', 'bem', 'antes', 'do', 'prazo', 'estipulado', '.'], tags=['0']),
 TaggedDocument(words=['Parabéns', 'lojas', 'Zoop', 'adorei', 'comprar', 'pela', 'Internet', 'seguro', 'e', 'prático', 'Parabéns', 'a', 'todos', 'feliz', 'Páscoa'], tags=['1']),
 TaggedDocument(words=['aparelho', 'eficiente', '.', 'no', 'site', 'a', 'marca', 'do', 'aparelho', 'esta', 'impresso', 'como', '3desinfector', 'e', 'ao', 'chegar', 'esta', 'com', 'outro', 'nome', '...', 'atualizar', 'com', 'a', 'marca', 'correta', 'uma', 'vez', 'que', 'é', 'o', 'mesmo', 'aparelho'], tags=['2']),
 TaggedDocument(words=['Mas', 'um', 'pouco', ',', 'travando', '...', 'pelo', 'valor', 'ta', 'Boa', '.'], tags=['3']),
 TaggedDocument(words=['Vendedor', 'confiável', ',', 'produto', 'ok', 'e', 'entrega', 'antes', 'do', 'prazo', '.'], tags=['4']),
 TaggedDocument(words=['GOSTARIA', 'DE', 'SABER', 'O', 'QUE', 'HOUVE', ',', 'SEMPRE', 'RECEBI', 'E', 'ESSA', 'COMPRA', 'AGORA', 'ME', 'DECPCIONOU'], tags=['5']),
 T

In [ ]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)

In [ ]:
vetor_inferido = modelo.infer_vector(['entrega'])
vetor_inferido

array([-0.10896794, -0.06701613,  0.06192388, -0.05586853,  0.08436941,
        0.06020607, -0.03059095, -0.02407885,  0.00976728, -0.10187825,
       -0.11544151,  0.02189551, -0.09198041,  0.03172444, -0.07342158,
       -0.07276786, -0.05136487,  0.02084706, -0.03089158,  0.01622137,
        0.02541097, -0.02914363,  0.08758707, -0.02851101, -0.01859125,
       -0.00670052, -0.0934583 , -0.07537099, -0.03776164, -0.05065462,
        0.06621411, -0.01816878,  0.02353206,  0.11650657,  0.03630283,
       -0.03388963, -0.11328775,  0.14241338, -0.10954304,  0.04181206,
        0.01305845,  0.06051464,  0.00907468, -0.01239336, -0.00919441,
       -0.1071564 ,  0.03907961, -0.00078529, -0.04537812,  0.00394392,
        0.00619319, -0.01932066,  0.0364631 , -0.03134329,  0.10149232,
       -0.11562458, -0.03065437, -0.02779194, -0.04036809, -0.06603919,
        0.02140157, -0.03385139,  0.01659509, -0.06770518,  0.02563674,
       -0.01192846,  0.06040177,  0.00086099,  0.02567412,  0.02

In [ ]:
frases_similares = modelo.dv.most_similar([vetor_inferido])
frases_similares

[('24884', 0.8574341535568237),
 ('24666', 0.8434157371520996),
 ('9304', 0.8364419937133789),
 ('33496', 0.832653820514679),
 ('30445', 0.8201867341995239),
 ('38093', 0.8191767930984497),
 ('19676', 0.8186330199241638),
 ('15297', 0.8182556629180908),
 ('35887', 0.8129003047943115),
 ('36551', 0.8096962571144104)]

In [ ]:
for idx, similaridade in frases_similares:
  print(f'Review: {dados.iloc[int(idx)]["review"]} - Similaridade: {similaridade:.4f}')

Review: entrega antes do prazo - Similaridade: 0.8574
Review: entrega antes do prazo - Similaridade: 0.8434
Review: entrega no prazo. - Similaridade: 0.8364
Review: Ainda estou esperando a entrega!  - Similaridade: 0.8327
Review: Comprem nas Zoop entrega certaaaa - Similaridade: 0.8202
Review: Ainda estou aguardando a entrega - Similaridade: 0.8192
Review: Muito boa entrega  - Similaridade: 0.8186
Review: Muito satisfeito com os produtos e entrega. - Similaridade: 0.8183
Review: Rapidez na entrega - Similaridade: 0.8129
Review: Compra tranquila e entrega sem problemas. - Similaridade: 0.8097


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords

def tratamento_inicial(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])

In [ ]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

In [ ]:
dados['tratamento_1']

,tratamento_1
0,recebi bem antes prazo estipulado
1,parabéns lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiável produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condições...
39458,produto não enviado nf não existe venda nf cer...
39459,excelente mochila entrega super rápida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.1 MB/s eta 0:00:00


In [ ]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [ ]:
dados['tratamento_2'] = sem_acentos
dados['tratamento_2']

,tratamento_2
0,recebi bem antes prazo estipulado
1,parabens lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiavel produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condicoes...
39458,produto nao enviado nf nao existe venda nf cer...
39459,excelente mochila entrega super rapida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [ ]:
dados['review_token'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in dados.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)
for idx, similaridade in frases_similares:
  print(f'Review: {dados.iloc[int(idx)]["tratamento_2"]} - Similaridade: {similaridade:.4f}')

Review: entrega antes prazo - Similaridade: 0.8823
Review: recebi produto antes prazo entrega intercorrencias - Similaridade: 0.8698
Review: entrega prazo produto acordo anuncio - Similaridade: 0.8694
Review: entrega prazo produto conforme solicitado - Similaridade: 0.8680
Review: prazo entrega gigante assim conseguiram atrasar entrega - Similaridade: 0.8669


In [ ]:
df = dados.drop_duplicates(subset='tratamento_2')
df

,nota_review,review,review_token,tratamento_1,tratamento_2
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo
...,...,...,...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...
39458,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...
39459,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...
39460,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [ ]:
df['tratamento_2'][3319]

'simplismeente ameiii'

In [ ]:
df = df[df['tratamento_2'] != '']

In [ ]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

/tmp/ipython-input-1534602499.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)


In [ ]:
df['tratamento_3'][3319]

'simplismente amei'

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
!pip install stanza
!python -m stanza.download pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 20.1 MB/s eta 0:00:00
/usr/bin/python3: No module named stanza.download


In [ ]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt',
                      processors='tokenize,lemma',
                      use_gpu=False,
                      batch_size=64,
                      n_process=4)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: pt (Portuguese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package         |
-------------------------------
| tokenize  | bosque          |
| mwt       | bosque          |
| lemma     | bosque_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
def lematizar_texto(textos):
  texto_lematizado = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizado.append(frase_lematizada)

  return texto_lematizado

In [ ]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

['gostar muito experiencia comprar',
 'meu filha gostar produto',
 'compra ser facil compra rapida']

In [ ]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])
df['tratamento_4']

/tmp/ipython-input-112385964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_4'] = lematizar_texto(df['tratamento_3'])


,tratamento_4
0,recebi bem antes prazo estipulado
1,parabem loja zop adorei comprar internet segur...
2,aparelho eficiente site marca aparelho imprimi...
3,pouco travar valor to bom
4,vendedor confiavel produto ok entregar antes p...
...,...
31469,entregar dentro prazo produto chegar condicoe ...
31470,produto nao enviado em f nao existir venda em ...
31471,excelente mochila entregar super rapida super ...
31472,solicitei compra capa retrovisor celta prisma ...


In [45]:
df_lem = df.drop_duplicates(subset='tratamento_4')
df_lem

,nota_review,review,review_token,tratamento_1,tratamento_2,tratamento_3,tratamento_4
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...,parabens lojas zop adorei comprar internet seg...,parabem loja zop adorei comprar internet segur...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho imprimi...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa,pouco travando valor ta boa,pouco travar valor to bom
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entregar antes p...
...,...,...,...,...,...,...,...
31469,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...,entregou dentro prazo produto chegou condicoes...,entregar dentro prazo produto chegar condicoe ...
31470,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado em f nao existir venda em ...
31471,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...,excelente mochila entrega super rapida super r...,excelente mochila entregar super rapida super ...
31472,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [46]:
df_lem.reset_index(drop=True, inplace=True)

In [49]:
df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in df_lem.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)
for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_4"]} - Similaridade: {similaridade:.4f}')

/tmp/ipython-input-763663982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)


Review: entrega demorando - Similaridade: 0.8961
Review: compra realizar dia 26 01 2018 hoje 26 02 2018 pruduto nao entregar - Similaridade: 0.8766
Review: comprei dois unidade apenas entregar - Similaridade: 0.8762
Review: vir faltando cadeira - Similaridade: 0.8762
Review: ultimamente atrasar entrega - Similaridade: 0.8756


In [50]:
modelo = Doc2Vec(vector_size=300, min_count=2,
                 window=5, workers=1, seed=45, epochs=20)
modelo.build_vocab(dados_tag)

modelo.train(dados_tag, total_examples=modelo.corpus_count,
             epochs=modelo.epochs)

In [51]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

Review: ultimamente atrasando entregas - Similaridade: 0.9042
Review: entregas demorando - Similaridade: 0.8839
Review: sempre bem atendido - Similaridade: 0.8822
Review: produtos certos achei deveriam entregues juntos compra achei pouco demorada entrega - Similaridade: 0.8773
Review: entrega atrasou - Similaridade: 0.8767


In [52]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['vendedor'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

Review: compraria novamente vendedor - Similaridade: 0.9263
Review: ótimo vendedor superou expectativas produto excelente - Similaridade: 0.9191
Review: ótimo vendedor - Similaridade: 0.9137
Review: exelente vendedor - Similaridade: 0.9100
Review: vendedor super - Similaridade: 0.9086


In [54]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['reputacao loja'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

Review: realmente enviaram prometeram chegou prazo determinado - Similaridade: 0.1675
Review: recebi produto solicitado prazo ridículo buscar bicicleta chegaria antes - Similaridade: 0.1585
Review: entregaram dentro prazo porém previsão extensa zoop sempre entregou antes prazo achei produto iria chegar antes previsto - Similaridade: 0.1556
Review: embora previsto prazo entrega 20 dias recebi antes disso parabéns - Similaridade: 0.1550
Review: produto chegou bem antes prazo previsto maneira pedi - Similaridade: 0.1531


In [55]:
modulo_url ='https://tfhub.dev/google/universal-sentence-encoder/4'

In [56]:
import tensorflow_hub as hub

modelo = hub.load(modulo_url)

In [57]:
modelo

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7cec9f2cee10>

In [59]:
reviews = df['tratamento_3'].tolist()
reviews_emb = modelo(reviews)
reviews_emb

<tf.Tensor: shape=(31474, 512), dtype=float32, numpy=
array([[ 0.04775169,  0.01094663, -0.0541508 , ...,  0.09078011,
        -0.02599917, -0.09261213],
       [-0.00746649, -0.06605157, -0.04505714, ...,  0.05561579,
        -0.02624756, -0.07801282],
       [-0.00820429,  0.04282852, -0.02907449, ...,  0.07496072,
        -0.06071885, -0.07618523],
       ...,
       [ 0.02074313, -0.0543616 , -0.06415588, ...,  0.0719423 ,
         0.0105929 , -0.07603392],
       [-0.04131037, -0.03054746,  0.01322938, ...,  0.06785525,
        -0.0252889 , -0.07106064],
       [ 0.05717909, -0.05808825, -0.02038886, ...,  0.07339787,
        -0.06409648, -0.08014238]], dtype=float32)>

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(reviews_emb)

array([[1.0000002 , 0.61586297, 0.5175547 , ..., 0.5427756 , 0.57541025,
        0.6691027 ],
       [0.61586297, 0.99999994, 0.5666107 , ..., 0.63920605, 0.6011834 ,
        0.6711744 ],
       [0.5175547 , 0.5666107 , 0.9999999 , ..., 0.58997375, 0.6564975 ,
        0.5922985 ],
       ...,
       [0.5427756 , 0.63920605, 0.58997375, ..., 1.        , 0.6378722 ,
        0.61628115],
       [0.57541025, 0.6011834 , 0.6564975 , ..., 0.6378722 , 1.0000001 ,
        0.6389235 ],
       [0.6691027 , 0.6711744 , 0.5922985 , ..., 0.61628115, 0.6389235 ,
        1.        ]], dtype=float32)

In [63]:
import numpy as np

def sentencas_similares(tema, reviews, reviews_emb, top_n=5):
  tema_emb = modelo([tema])
  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()

  indices_similares = np.argsort(-similaridades)
  for idx in indices_similares[:top_n]:
    print(f'Review: {reviews[idx]} - Similaridade: {similaridades[idx]:.4f}')

In [64]:
sentencas_similares('entrega', reviews, reviews_emb)

Review: eficiente entrega - Similaridade: 0.6829
Review: entrega eficiente - Similaridade: 0.6756
Review: agil entrega - Similaridade: 0.6563
Review: entrega prazo - Similaridade: 0.6473
Review: entrega agil - Similaridade: 0.6471


In [65]:
sentencas_similares('reputação loja', reviews, reviews_emb, top_n=10)

Review: otima loja - Similaridade: 0.8578
Review: recomento loja - Similaridade: 0.8339
Review: pessima loja - Similaridade: 0.8335
Review: adorei loja - Similaridade: 0.8170
Review: rapidez eficiencia loja - Similaridade: 0.8018
Review: loja confianca - Similaridade: 0.7909
Review: produto loja confianca - Similaridade: 0.7888
Review: loja ruim - Similaridade: 0.7868
Review: otimas experiencias loja - Similaridade: 0.7839
Review: satisfeita produto loja - Similaridade: 0.7816


In [71]:
!pip install gradio

In [72]:
import gradio as gr

In [73]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

modulo_url ='https://tfhub.dev/google/universal-sentence-encoder/4'
modelo = hub.load(modulo_url)

def tratar_texto(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  word_tokenize_pt = partial(word_tokenize, language='portuguese')
  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')
  tokens = [w for w in tokens if w not in stop_words]

  texto_sem_acentos = unidecode.unidecode(' '.join(tokens))

  texto_normalizado = re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto_sem_acentos)

  return texto_normalizado

def process_csv(arquivo, tema):
  df = pd.read_csv(arquivo.name)

  df['review_tratada'] = df['review'].apply(tratar_texto)
  df = df[df['review_tratada']!='']
  df.drop_duplicates(subset='review_tratada', inplace=True)

  reviews_emb = modelo(df['review_tratada'].tolist())
  tema_emb = modelo([tema])

  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()
  top_indices = np.argsort(-similaridades)

  similar_reviews = df[['nota_review', 'review']].iloc[top_indices]
  similar_df = similar_reviews.head(200)
  nome_arquivo = f'reviews_similares_{tema}.csv'
  similar_reviews.to_csv(nome_arquivo)

  return similar_df, nome_arquivo

with gr.Blocks() as app:
  with gr.Row():
    gr.Markdown('## Encontrando as reviews mais similares ao tema')
  csv_entrada = gr.File(label='Envie o CVS com as reviews', file_types=['.csv'])
  tema_entrada = gr.Textbox(label='Digite o tema de busca (Ex.: "Entrega")')

  botao = gr.Button('Clique para buscas as reviews')

  tabela_saida = gr.Dataframe(label='Top 200 reviews similares', headers=['Nota', 'Reviews'], interactive=False)
  arquivo_saida = gr.File(label='Baixar CSV ordenado com as reviews mais similares ao tema', interactive=False)

  botao.click(process_csv, inputs=[csv_entrada, tema_entrada], outputs=[tabela_saida, arquivo_saida])
app.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73032d4f987c84db02.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [70]:
process_csv(url, 'entrega')

,nota_review,review
4623,5,Muito eficiente a entrega.
5432,5,Entrega eficiente.
24922,5,Muito ágil na entrega.
34,3,Entrega no prazo
8569,5,Entrega foi muito ágil
...,...,...
27152,3,Os nichos vieram com um péssimo acabamento.
13502,2,Me decepcionei com o produto. O material é de ...
33888,1,Estou solicitando a devolução do produto.
35473,1,Faltou a FLANGE da cadeira que e considerada a...
